# Data Filter
### The purpose of this notebook is four-fold:
1) Filter data to only the relevant rows

2) Delete the unnecessary columns

3) Suitably edit the text to allow for topic modeling

4) Create new variables to assist with demographic comparisons of topics


### Note: 
URL: https://github.com/rudeboybert/JSE_OkCupid

At 141 MB (unzipped) and 53 MB (zipped), this is too large to be uploaded onto Github.
This notebook is only required to convert the original data to a manageable size. 
If you already have access to the filtered data, 'compressed_okcupid.csv', then you may proceed to the next steps in the notebooks

In [7]:
!pip install pyspellchecker

In [3]:
#!pip install pyenchant==1.6.6
!pip install compound-word-splitter
!pip install pyspellchecker
!pip install wordninja

In [23]:
#General Purpose Imports
import numpy as np
import pandas as pd
import warning
import re
from spellchecker import SpellChecker
warnings.filterwarnings('ignore')
from tqdm import tqdm
tqdm.pandas()
import wordninja

In [9]:
spell = SpellChecker(distance=2)
lolly = ['I', 'love', 'cantilever', 'pecocks']
spell.correction('greatbutt')

'greatbutt'

In [ ]:
import re
corrections ={r"won\'t": "will not",
              r"can\'t": "can not",
              r"n\'t": " not",
              r"\'re": " are",
              r"\'s": " is",
              r"\'d": " would",
              r"\'ll": " will",
              r"\'t": " not",
              r"\'ve": " have",
              r"\'m": " am"}

def decontracted(phrase):  
    # Taken from:
    # https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
    # specific
    for k,v in corrections.items():
        phrase = re.sub(k, v, phrase)
    return phrase

def split_word(compound_word):
    '''
    Takes in compound word
    Splits it into individual words
    Returns string with spaced words
    '''
    sep_words = wordninja.split(compound_word)
    print('The separated words are {}'.format(sep_words))
    cleaned = ' '.join(sep_words)
    print(cleaned)
    return cleaned

def decide_split(word):
    spellcheck = SpellChecker()
    if not spellcheck[word]:
        nearest = spellcheck.correction(word)
        #When there is no valid word, the nearest word
        #is the same as the original
        if word == nearest:
            #print('The compound word is {}'.format(word))
            return split_word(word)
        else:
            #print('The accepted word is {}'.format(word))            
            #print(nearest)
            return nearest
    else:
        return word

def split_incorrect(text, punctuation=True):
    '''
    Takes in a long string
    The punctuation parameter checks if punctuation marks are to
    be preserved
    Splits into component words, checks for incorrect spellings
    For incorrect spellings, checks if is possibly compound
    If not, then looks for the closest one word in the dictionary
    Returns the entire text with all words corrected
    '''
    #if punctuation:

    cleaned_words = []
    words= re.split('\.+', text)
    #print(words)
    for word in words:
        word = decontracted(word)
        checked_word = decide_split(word)
        print('The checked word is {}'.format(checked_word))
        cleaned_words.append(checked_word)
    final = " ".join(cleaned_words)
    print("done!")
    return final
    
test_text = 'John had a big cock, it crowedatsunrise every morning'
real_text = 'i really like meeting new people. small-world networks fascinateme.beer is important. without beer, there would be nocivilization.somehow i can only sing spanish, but not speak it. i suppose musicis the universal language?coffee enthusiast.why am i the only one singing when riding my bike around themission?'
real_text_2 = "i'm an adventurer first, i take calculatedrisks whenever it seems fun, and i always take care of those aroundme.i'm a writer, astory teller, a photographer and those things arebasically the same thing to me. i love playing with lights, sound,and really any machine that has locomotion or pretty blinky thingson it. love driving, racing, flying, sailing, sky diving, and adrenaline issacred. that said, i so appreciate a night at home with a goodbook, bottle of wine, or if i have company, a movie andconversation. not at the same time, though, i like to hear themovie, then focus on you:) more later..."
#print(split_incorrect(test_text))


In [ ]:
%timeit print(split_incorrect(test_text))

In [20]:
import wordninja
#wordninja.split('hammerandtongs')

def split_word(compound_word):
    '''
    Takes in compound word
    Splits it into individual words
    Returns string with spaced words
    '''
    sep_words = wordninja.split(compound_word)
    cleaned = ' '.join(sep_words)
    return cleaned

split_word('guzzlpetrl')
    
    

'gu zz l petr l'

In [6]:
#Imports for Text Analytics
from bs4 import BeautifulSoup
import spacy
import textacy
nlp = spacy.load('en_core_web_sm')
en = textacy.load_spacy_lang("en_core_web_sm")
from textacy import TextStats

In [20]:
df = pd.read_csv('D:\Dropbox\Repositories\persp-research_S19\okcupid\data\profiles.csv')
must_haves = ['body_type', 'height', 'education', 'ethnicity', 'sex', 'essay0', 'essay9']
df = df.dropna(subset= must_haves)
df = df[(df['sex']=="m")&(df['orientation']=="straight")]
df = df.drop(columns=['essay1', 'essay2', 'essay3','essay4','essay5','essay6','essay7',
                      'essay8', 'essay9', 'income','job','last_online','location','offspring',
                      'orientation','pets','religion','sex','sign','smokes','speaks','status',
                      'diet', 'drinks', 'drugs'])

### CREATING NEW COLUMNS


Many of the sections here are taken directly from the following link, with specific modifications
Taken directly from:
https://github.com/UM-CSS/CSSLabs-NLP/blob/master/1_Data_munging.ipynb

In [18]:
def recode(text, dictionary, default=np.nan):
    '''Function for recoding categories in a column based on exact matches'''
    out = default
    text = str(text)
    
    for x in dictionary.keys():
        for y in dictionary[x]:
            if y == text: #exact match
                out = x
                return out
    return out

def recode_fuzzy(text, dictionary, default=np.nan):
    '''Function for recoding categories in a column based on partial matches'''
    out = default
    text = str(text)
    
    for x in dictionary.keys():
        for y in dictionary[x]:
            if y in text: #partial match
                out = x
                return out
    return out

In [19]:
ed_levels = {'High School or less': ['dropped out of high school', 'working on high school','graduated from high school', 'working on college/university', 
                    'two-year college', 'dropped out of college/university', 
                    'high school'], 
             'More than High School': ['graduated from college/university', 
                    'working on masters program', 'working on ph.d program', 
                    'college/university', 'working on law school', 
                    'dropped out of masters program', 
                    'dropped out of ph.d program', 'dropped out of law school', 
                    'dropped out of med school',
                    'graduated from masters program',
                    'graduated from ph.d program',                           
                    'graduated from law school', 
                    'graduated from med school', 'masters program', 
                    'ph.d program', 'law school', 'med school']}

#body type
bodies = {'fit': ['fit', 'athletic', 'jacked'], 
          'not_fit': ['average', 'thin', 'skinny','curvey', 'a little extra', 
                      'full figured', 'overweight', 'rather not say', 'used up']
         }

In [20]:
df['edu'] = df.education.apply(recode, dictionary=ed_levels, 
                                            default='unknown')
df['fit'] = df.body_type.apply(recode, dictionary=bodies, 
                                            default='unknown')

In [21]:
# race/ethnicity for exact matching
ethn = {'White': ['white', 'middle eastern', 'middle eastern, white'], 
        'Asian': ['asian', 'indian', 'asian, pacific islander'], 
        'Black': ['black']       }   

# race/ethnicityfor fuzzy matching
ethn2 = {'Latinx': ['latin'], 'multiple': [','], np.nan: ['nan']}

In [22]:
def census_2010_ethnicity(t):
    text = str(t)
    e = recode(text, ethn, default='other')
    if 'other' == e:
        e = recode_fuzzy(text, ethn2, default='other')
    return e

df['race_ethnicity'] = df.ethnicity.apply(census_2010_ethnicity)

In [23]:
def height_check(inches):
    h = 'not_short'
    if inches <= 69:
        h = 'short'
    return h
df['height'] = pd.to_numeric(df['height'])
df['height_group'] = df.height.apply(height_check)

## TEXT EDITING

In [24]:
# Some of the essays have just a link in the text. BeautifulSoup sees that and gets 
# the wrong idea. This line hides those warnings.
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
def clean(text):
    if pd.isnull(text):
        t = np.nan
    else:
        t = BeautifulSoup(text, 'lxml').get_text()
        t = t.lower()
        t = t.strip().replace('\n','').replace("\r", " ").replace('\t', '')
        bad_words = ['http', 'www', '\nnan']

        for b in bad_words:
            t = t.replace(b, '')
    if t == '':
        t = np.nan
    
    return t

In [25]:
#Clearing out all HTML and unnecessary characters
df['essay0'] =df['essay0'].progress_apply(clean)
df['essay9'] =df['essay9'].progress_apply(clean)


100%|██████████████████████████████████████████████████████████████████████████| 18831/18831 [00:05<00:00, 3622.74it/s]

In [33]:
#These functions help assess the level of complexity
def get_flesch(text):    
    doc = textacy.make_spacy_doc(text, lang=en)
    ts = TextStats(doc)
    try:
        return ts.flesch_kincaid_grade_level
    except ZeroDivisionError:
        return (11.8 * ts.n_syllables) + (0.39 * ts.n_words) - 15.59

def get_npoly(text):
    doc = textacy.make_spacy_doc(text, lang=en)
    ts = TextStats(doc)
    return ts.n_polysyllable_words

In [28]:
df['long_words'] = df['essay0'].progress_apply(get_npoly)



  0%|                                                                                        | 0/18831 [00:00<?, ?it/s]

 12%|█████████▍                                                                   | 2305/18831 [01:19<07:17, 37.80it/s]

  0%|                                                                                | 7/18831 [00:00<09:53, 31.72it/s]

  0%|                                                                               | 11/18831 [00:00<09:46, 32.11it/s]

  0%|                                                                               | 15/18831 [00:00<09:21, 33.53it/s]

  0%|                                                                               | 19/18831 [00:00<08:58, 34.96it/s]

  0%|                                                                               | 23/18831 [00:00<08:43, 35.94it/s]

  0%|                                                                               | 29/18831 [00:00<08:13, 38.13it/s]

  0%|▏                        

  3%|██▌                                                                           | 625/18831 [00:17<11:27, 26.49it/s]

  3%|██▌                                                                           | 629/18831 [00:17<10:57, 27.70it/s]

  3%|██▌                                                                           | 632/18831 [00:17<11:58, 25.34it/s]

  3%|██▋                                                                           | 635/18831 [00:17<11:39, 26.02it/s]

  3%|██▋                                                                           | 640/18831 [00:18<10:10, 29.80it/s]

  3%|██▋                                                                           | 644/18831 [00:18<09:39, 31.36it/s]

  3%|██▋                                                                           | 648/18831 [00:18<09:18, 32.55it/s]

  3%|██▋                                                                           | 652/18831 [00:18<09:09, 33.10it/s]

  3%|██▋                        

  7%|█████                                                                        | 1237/18831 [00:35<10:22, 28.28it/s]

  7%|█████                                                                        | 1241/18831 [00:35<10:06, 28.98it/s]

  7%|█████                                                                        | 1245/18831 [00:35<09:21, 31.29it/s]

  7%|█████                                                                        | 1250/18831 [00:35<09:17, 31.52it/s]

  7%|█████▏                                                                       | 1257/18831 [00:36<07:53, 37.14it/s]

  7%|█████▏                                                                       | 1264/18831 [00:36<07:01, 41.69it/s]

  7%|█████▏                                                                       | 1269/18831 [00:36<06:49, 42.86it/s]

  7%|█████▏                                                                       | 1274/18831 [00:36<07:23, 39.57it/s]

  7%|█████▏                     

 10%|███████▊                                                                     | 1906/18831 [00:53<08:16, 34.08it/s]

 10%|███████▊                                                                     | 1910/18831 [00:53<08:23, 33.59it/s]

 10%|███████▊                                                                     | 1914/18831 [00:53<08:08, 34.65it/s]

 10%|███████▊                                                                     | 1920/18831 [00:53<07:18, 38.53it/s]

 10%|███████▉                                                                     | 1927/18831 [00:53<06:22, 44.20it/s]

 10%|███████▉                                                                     | 1932/18831 [00:53<06:40, 42.20it/s]

 10%|███████▉                                                                     | 1937/18831 [00:54<06:50, 41.11it/s]

 10%|███████▉                                                                     | 1942/18831 [00:54<06:44, 41.79it/s]

 10%|███████▉                   

 14%|██████████▍                                                                  | 2548/18831 [01:11<07:31, 36.07it/s]

 14%|██████████▍                                                                  | 2552/18831 [01:11<07:19, 37.06it/s]

 14%|██████████▍                                                                  | 2556/18831 [01:11<08:04, 33.60it/s]

 14%|██████████▍                                                                  | 2560/18831 [01:11<08:20, 32.54it/s]

 14%|██████████▍                                                                  | 2565/18831 [01:11<07:54, 34.25it/s]

 14%|██████████▌                                                                  | 2570/18831 [01:11<07:16, 37.23it/s]

 14%|██████████▌                                                                  | 2578/18831 [01:11<06:10, 43.91it/s]

 14%|██████████▌                                                                  | 2583/18831 [01:11<06:38, 40.78it/s]

 14%|██████████▌                

...

 95%|████████████████████████████████████████████████████████████████████████▍   | 17963/18831 [08:26<00:20, 41.62it/s]

 95%|████████████████████████████████████████████████████████████████████████▌   | 17968/18831 [08:26<00:35, 24.61it/s]

 95%|████████████████████████████████████████████████████████████████████████▌   | 17973/18831 [08:26<00:31, 27.27it/s]

 95%|████████████████████████████████████████████████████████████████████████▌   | 17977/18831 [08:27<00:29, 29.23it/s]

 95%|████████████████████████████████████████████████████████████████████████▌   | 17982/18831 [08:27<00:26, 32.10it/s]

 96%|████████████████████████████████████████████████████████████████████████▌   | 17987/18831 [08:27<00:24, 35.06it/s]

 96%|████████████████████████████████████████████████████████████████████████▌   | 17993/18831 [08:27<00:22, 37.78it/s]

 96%|████████████████████████████████████████████████████████████████████████▋   | 17998/18831 [08:27<00:23, 35.94it/s]

 96%|███████████████████████████

 99%|███████████████████████████████████████████████████████████████████████████▏| 18641/18831 [08:44<00:07, 26.51it/s]

 99%|███████████████████████████████████████████████████████████████████████████▏| 18644/18831 [08:44<00:07, 25.44it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 18649/18831 [08:44<00:06, 28.27it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 18653/18831 [08:45<00:06, 28.66it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 18658/18831 [08:45<00:05, 30.58it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 18662/18831 [08:45<00:05, 30.99it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 18667/18831 [08:45<00:04, 34.26it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 18671/18831 [08:45<00:05, 29.60it/s]

 99%|███████████████████████████

  2%|█▋                                                                            | 405/18831 [00:11<09:17, 33.07it/s]

  2%|█▋                                                                            | 409/18831 [00:11<09:13, 33.31it/s]

  2%|█▋                                                                            | 413/18831 [00:11<09:30, 32.27it/s]

  2%|█▋                                                                            | 417/18831 [00:11<09:08, 33.56it/s]

  2%|█▋                                                                            | 421/18831 [00:12<09:04, 33.83it/s]

  2%|█▊                                                                            | 427/18831 [00:12<08:07, 37.75it/s]

  2%|█▊                                                                            | 431/18831 [00:12<08:36, 35.63it/s]

  2%|█▊                                                                            | 435/18831 [00:12<09:18, 32.92it/s]

  2%|█▊                         

ZeroDivisionError: float division by zero



  4%|██▋                                                                           | 663/18831 [00:29<09:43, 31.14it/s]

In [30]:
df['flesch'] = df['essay0'].progress_apply(get_flesch)




  0%|                                                                                        | 0/18831 [00:00<?, ?it/s]


  0%|                                                                                | 4/18831 [00:00<07:50, 40.00it/s]


  0%|                                                                                | 7/18831 [00:00<09:51, 31.84it/s]


  0%|                                                                               | 11/18831 [00:00<09:24, 33.32it/s]


  0%|                                                                               | 14/18831 [00:00<10:09, 30.86it/s]


  0%|                                                                               | 18/18831 [00:00<09:34, 32.72it/s]


  0%|                                                                               | 22/18831 [00:00<09:04, 34.52it/s]


  0%|                                                                               | 27/18831 [00:00<08:14, 38.06it/s]


  0%|▏               

  3%|██▌                                                                           | 610/18831 [00:17<09:51, 30.79it/s]


  3%|██▌                                                                           | 614/18831 [00:17<09:38, 31.51it/s]


  3%|██▌                                                                           | 618/18831 [00:18<09:20, 32.51it/s]


  3%|██▌                                                                           | 622/18831 [00:18<08:51, 34.26it/s]


  3%|██▌                                                                           | 626/18831 [00:18<10:50, 27.99it/s]


  3%|██▌                                                                           | 630/18831 [00:18<10:23, 29.21it/s]


  3%|██▋                                                                           | 634/18831 [00:18<10:16, 29.53it/s]


  3%|██▋                                                                           | 638/18831 [00:18<10:34, 28.67it/s]


  3%|██▋                

  7%|█████▏                                                                       | 1262/18831 [00:35<06:52, 42.56it/s]


  7%|█████▏                                                                       | 1267/18831 [00:35<06:34, 44.55it/s]


  7%|█████▏                                                                       | 1272/18831 [00:36<07:11, 40.67it/s]


  7%|█████▏                                                                       | 1277/18831 [00:36<07:56, 36.80it/s]


  7%|█████▏                                                                       | 1281/18831 [00:36<09:16, 31.55it/s]


  7%|█████▎                                                                       | 1285/18831 [00:36<10:49, 27.00it/s]


  7%|█████▎                                                                       | 1288/18831 [00:36<11:07, 26.30it/s]


  7%|█████▎                                                                       | 1294/18831 [00:36<09:15, 31.58it/s]


  7%|█████▎             

 10%|███████▊                                                                     | 1907/18831 [00:53<07:32, 37.41it/s]


 10%|███████▊                                                                     | 1911/18831 [00:54<08:14, 34.23it/s]


 10%|███████▊                                                                     | 1916/18831 [00:54<07:35, 37.14it/s]


 10%|███████▊                                                                     | 1923/18831 [00:54<06:32, 43.06it/s]


 10%|███████▉                                                                     | 1930/18831 [00:54<06:00, 46.83it/s]


 10%|███████▉                                                                     | 1936/18831 [00:54<06:43, 41.84it/s]


 10%|███████▉                                                                     | 1941/18831 [00:54<06:34, 42.86it/s]


 10%|███████▉                                                                     | 1946/18831 [00:54<06:24, 43.96it/s]


 10%|███████▉           

 14%|██████████▍                                                                  | 2561/18831 [01:12<07:47, 34.84it/s]


 14%|██████████▍                                                                  | 2565/18831 [01:12<07:28, 36.24it/s]


 14%|██████████▌                                                                  | 2570/18831 [01:12<06:53, 39.31it/s]


 14%|██████████▌                                                                  | 2578/18831 [01:12<05:50, 46.31it/s]


 14%|██████████▌                                                                  | 2584/18831 [01:12<06:10, 43.83it/s]


 14%|██████████▌                                                                  | 2589/18831 [01:13<06:27, 41.86it/s]


 14%|██████████▌                                                                  | 2594/18831 [01:13<06:51, 39.43it/s]


 14%|██████████▋                                                                  | 2599/18831 [01:13<07:32, 35.85it/s]


 14%|██████████▋        

...

 86%|█████████████████████████████████████████████████████████████████▌          | 16243/18831 [07:54<01:16, 33.82it/s]


 86%|█████████████████████████████████████████████████████████████████▌          | 16247/18831 [07:54<01:18, 32.93it/s]


 86%|█████████████████████████████████████████████████████████████████▌          | 16251/18831 [07:54<01:17, 33.30it/s]


 86%|█████████████████████████████████████████████████████████████████▌          | 16255/18831 [07:54<01:14, 34.42it/s]


 86%|█████████████████████████████████████████████████████████████████▌          | 16259/18831 [07:54<01:14, 34.44it/s]


 86%|█████████████████████████████████████████████████████████████████▋          | 16263/18831 [07:54<01:24, 30.24it/s]


 86%|█████████████████████████████████████████████████████████████████▋          | 16267/18831 [07:54<01:21, 31.32it/s]


 86%|█████████████████████████████████████████████████████████████████▋          | 16272/18831 [07:54<01:18, 32.78it/s]


 86%|███████████████████

 89%|███████████████████████████████████████████████████████████████████▉        | 16847/18831 [08:12<01:01, 32.51it/s]


 89%|████████████████████████████████████████████████████████████████████        | 16852/18831 [08:12<00:59, 33.01it/s]


 90%|████████████████████████████████████████████████████████████████████        | 16856/18831 [08:12<00:58, 33.69it/s]


 90%|████████████████████████████████████████████████████████████████████        | 16862/18831 [08:12<00:55, 35.43it/s]


 90%|████████████████████████████████████████████████████████████████████        | 16866/18831 [08:13<00:53, 36.59it/s]


 90%|████████████████████████████████████████████████████████████████████        | 16870/18831 [08:13<00:58, 33.75it/s]


 90%|████████████████████████████████████████████████████████████████████        | 16874/18831 [08:13<01:02, 31.33it/s]


 90%|████████████████████████████████████████████████████████████████████        | 16878/18831 [08:13<01:03, 30.52it/s]


 90%|███████████████████

 93%|██████████████████████████████████████████████████████████████████████▌     | 17492/18831 [08:30<00:37, 35.28it/s]


 93%|██████████████████████████████████████████████████████████████████████▌     | 17497/18831 [08:30<00:36, 36.89it/s]


 93%|██████████████████████████████████████████████████████████████████████▋     | 17501/18831 [08:30<00:35, 37.45it/s]


 93%|██████████████████████████████████████████████████████████████████████▋     | 17506/18831 [08:30<00:33, 39.93it/s]


 93%|██████████████████████████████████████████████████████████████████████▋     | 17511/18831 [08:30<00:31, 42.39it/s]


 93%|██████████████████████████████████████████████████████████████████████▋     | 17516/18831 [08:30<00:31, 41.74it/s]


 93%|██████████████████████████████████████████████████████████████████████▋     | 17523/18831 [08:31<00:28, 46.46it/s]


 93%|██████████████████████████████████████████████████████████████████████▋     | 17528/18831 [08:31<00:29, 44.08it/s]


 93%|███████████████████

 96%|█████████████████████████████████████████████████████████████████████████▏  | 18130/18831 [08:48<00:19, 35.65it/s]


 96%|█████████████████████████████████████████████████████████████████████████▏  | 18135/18831 [08:48<00:18, 38.11it/s]


 96%|█████████████████████████████████████████████████████████████████████████▏  | 18140/18831 [08:48<00:22, 31.21it/s]


 96%|█████████████████████████████████████████████████████████████████████████▏  | 18144/18831 [08:48<00:21, 32.51it/s]


 96%|█████████████████████████████████████████████████████████████████████████▏  | 18149/18831 [08:48<00:19, 35.17it/s]


 96%|█████████████████████████████████████████████████████████████████████████▎  | 18155/18831 [08:49<00:19, 34.90it/s]


 96%|█████████████████████████████████████████████████████████████████████████▎  | 18159/18831 [08:49<00:18, 36.00it/s]


 96%|█████████████████████████████████████████████████████████████████████████▎  | 18163/18831 [08:49<00:19, 34.34it/s]


 96%|███████████████████

100%|███████████████████████████████████████████████████████████████████████████▊| 18797/18831 [09:06<00:00, 39.87it/s]


100%|███████████████████████████████████████████████████████████████████████████▉| 18802/18831 [09:06<00:00, 39.72it/s]


100%|███████████████████████████████████████████████████████████████████████████▉| 18807/18831 [09:06<00:00, 39.24it/s]


100%|███████████████████████████████████████████████████████████████████████████▉| 18811/18831 [09:06<00:00, 36.74it/s]


100%|███████████████████████████████████████████████████████████████████████████▉| 18815/18831 [09:06<00:00, 35.46it/s]


100%|███████████████████████████████████████████████████████████████████████████▉| 18821/18831 [09:06<00:00, 39.08it/s]


100%|███████████████████████████████████████████████████████████████████████████▉| 18827/18831 [09:06<00:00, 42.17it/s]


100%|████████████████████████████████████████████████████████████████████████████| 18831/18831 [09:07<00:00, 34.42it/s]

In [38]:
df['long_words'].describe()

count    18831.000000
mean        11.322819
std         13.282247
min          0.000000
25%          3.000000
50%          8.000000
75%         15.000000
max        446.000000
Name: long_words, dtype: float64

In [39]:
df['flesch'].describe()

count    18831.000000
mean         7.282738
std          4.784808
min        -15.590000
25%          4.853526
50%          6.726154
75%          8.955000
max        268.000000
Name: flesch, dtype: float64

In [ ]:
df['words_split'] = df['essay0'].progress_apply(split_incorrect)


  1%|▌                                                                          | 135/18831 [44:20<37:07:22,  7.15s/it]


  1%|█                                                                        | 271/18831 [1:07:18<14:09:46,  2.75s/it]


  2%|█▌                                                                       | 407/18831 [1:31:34<63:17:50, 12.37s/it]


  3%|██                                                                       | 543/18831 [1:48:49<32:57:30,  6.49s/it]


  4%|██▋                                                                      | 679/18831 [2:16:04<29:30:39,  5.85s/it]


...

 39%|███████████████████████████▋                                           | 7343/18831 [35:41:13<91:09:11, 28.56s/it]


 40%|████████████████████████████▏                                          | 7479/18831 [36:03:38<33:54:14, 10.75s/it]


 40%|████████████████████████████▋                                          | 7615/18831 [36:29:40<60:18:14, 19.36s/it]


 41%|█████████████████████████████▏                                         | 7751/18831 [37:31:28<33:29:32, 10.88s/it]


 42%|█████████████████████████████▌                                         | 7827/18831 [39:35:00<38:01:02, 12.44s/it]

In [35]:
df.to_csv('compressed_okcupid.csv')